<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/ti_edgeai_tidl_tools_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EdgeAI TIDL Tools - THIS IS NOT VALIDATED YET, AS the Colan Runtime is bydefault starting with 3.7

## Introduction
This notebooks follows the steps mentioned in [edgeai-tidl-tools](https://github.com/TexasInstruments/edgeai-tidl-tools) github repository and prepares PC environment for Model compilation and PC emulation of inference. The compiled models can be copied to EVM/SK board for validation/benchmarking of DL inference on the target device.

## Prepare Colab Runtime

The default the version used by colab instance is ubuntu 18.04 with python 3.7. The below sections sets up right python 3.6 and pip3 versions


In [1]:
!sudo update-alternatives --set python3 /usr/bin/python3.6
!curl https://bootstrap.pypa.io/pip/3.6/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

update-alternatives: using /usr/bin/python3.6 to provide /usr/bin/python3 (python3) in manual mode
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2108k  100 2108k    0     0  11.5M      0 --:--:-- --:--:-- --:--:-- 11.5M
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.2 MB/s            
     |████████████████████████████████| 952 kB 38.2 MB/s            


## Clone and Setup Edgai-TIDL-Tools

The below setup is configured for J7 device with python3 API examples only. For CPP example refer documentation in the repo and update the setup command accordingly

In [2]:
!git clone https://github.com/TexasInstruments/edgeai-tidl-tools.git
!export DEVICE=j7 && cd edgeai-tidl-tools && source ./setup.sh  --skip_arm_gcc_download --skip_cpp_deps

Cloning into 'edgeai-tidl-tools'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (460/460), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 2166 (delta 349), reused 380 (delta 306), pack-reused 1706
Receiving objects: 100% (2166/2166), 10.48 MiB | 14.28 MiB/s, done.
Resolving deltas: 100% (1335/1335), done.
X64 Architecture
Installing python packages...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/kumardesappan/caffe2onnx to /tmp/pip-req-build-vj30y7g9
  Running command git clone --filter=blob:none -q https://github.com/kumardesappan/caffe2onnx /tmp/pip-req-build-vj30y7g9
  Resolved https://github.com/kumardesappan/caffe2onnx to commit b7e73feed3bbc5ddbdf25b87af93a2bae596055d
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 2.2 MB 4.0 MB/s            
     |████████████████████████████████| 35.9 MB 69 kB/s              
   

In [ ]:
import tflite_runtime.interpreter as tflite
import time
import os
import sys
import numpy as np
import PIL
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import re
import platform
os.environ["TIDL_TOOLS_PATH"]= "/content/edgeai-tidl-tools/tidl_tools"
os.environ["LD_LIBRARY_PATH"]= "/content/edgeai-tidl-tools/tidl_tools"
os.environ["TIDL_RT_PERFSTATS"] = "1"
os.environ["DEVICE"] = "j7"

sys.path.append("/content/edgeai-tidl-tools/examples/osrt_python")
from common_utils import *
tidl_tools_path = os.environ["TIDL_TOOLS_PATH"]

required_options = {
"tidl_tools_path":"/content/edgeai-tidl-tools/tidl_tools",
"artifacts_folder":"/content/edgeai-tidl-tools/model-artifacts",
}
output_images_folder = "/content/edgeai-tidl-tools/outputs"

calib_images = ['/content/edgeai-tidl-tools/test_data/airshow.jpg',
                '/content/edgeai-tidl-tools/test_data/ADE_val_00001801.jpg']
class_test_images = ['/content/edgeai-tidl-tools/test_data/airshow.jpg']
od_test_images    = ['/content/edgeai-tidl-tools/test_data/ADE_val_00001801.jpg']
seg_test_images   = ['/content/edgeai-tidl-tools/test_data/ADE_val_00001801.jpg']
DEVICE = os.environ["DEVICE"]

In [ ]:
def infer_image(interpreter, image_files, config):
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  floating_model = input_details[0]['dtype'] == np.float32
  batch = input_details[0]['shape'][0]
  height = input_details[0]['shape'][1]
  width  = input_details[0]['shape'][2]
  channel = input_details[0]['shape'][3]
  new_height = height  #valid height for modified resolution for given network
  new_width = width  #valid width for modified resolution for given network
  imgs    = []
 # copy image data in input_data if num_batch is more than 1 
  shape = [batch, new_height, new_width, channel]
  input_data = np.zeros(shape)
  for i in range(batch):
      imgs.append(Image.open(image_files[i]).convert('RGB').resize((new_width, new_height), PIL.Image.LANCZOS))
      temp_input_data = np.expand_dims(imgs[i], axis=0)  
      input_data[i] = temp_input_data[0] 
  if floating_model:
    input_data = np.float32(input_data)
    for mean, scale, ch in zip(config['mean'], config['std'], range(input_data.shape[3])):
        input_data[:,:,:, ch] = ((input_data[:,:,:, ch]- mean) * scale)
  else:
    input_data = np.uint8(input_data)
    config['mean'] = [0, 0, 0]
    config['std']  = [1, 1, 1]

  interpreter.resize_tensor_input(input_details[0]['index'], [batch, new_height, new_width, channel])
  interpreter.allocate_tensors()
  interpreter.set_tensor(input_details[0]['index'], input_data)
  

  interpreter.invoke()

  outputs = [interpreter.get_tensor(output_detail['index']) for output_detail in output_details]
  return imgs, outputs, new_height, new_width

In [ ]:
tf_models_configs = {
    # TFLite RT OOB Models
    'cl-tfl-mobilenet_v1_1.0_224' : {
        'model_path' : os.path.join("/content/edgeai-tidl-tools/models", 'mobilenet_v1_1.0_224.tflite'),
        'source' : {'model_url': 'http://software-dl.ti.com/jacinto7/esd/modelzoo/latest/models/vision/classification/imagenet1k/tf1-models/mobilenet_v1_1.0_224.tflite', 'opt': True},
        'mean': [127.5, 127.5, 127.5],
        'std' : [1/127.5, 1/127.5, 1/127.5],
        'num_images' : 3,
        'num_classes': 1001,
        'session_name' : 'tflitert',
        'model_type': 'classification'
    }
}

model = 'cl-tfl-mobilenet_v1_1.0_224'
download_model(tf_models_configs, model)
config = tf_models_configs[model]

if config['model_type'] == 'classification':
    test_images = class_test_images
elif config['model_type'] == 'od':
    test_images = od_test_images
elif config['model_type'] == 'seg':
    test_images = seg_test_images

#set delegate options
delegate_options = {}
delegate_options.update(required_options)
delegate_options.update(optional_options)
# stripping off the ss-tfl- from model namne
delegate_options['artifacts_folder'] = delegate_options['artifacts_folder'] + '/' + model + '/'

if config['model_type'] == 'od':
    delegate_options['object_detection:meta_layers_names_list'] = config['meta_layers_names_list'] if ('meta_layers_names_list' in config) else ''
    delegate_options['object_detection:meta_arch_type'] = config['meta_arch_type'] if ('meta_arch_type' in config) else -1


os.makedirs(delegate_options['artifacts_folder'], exist_ok=True)
for root, dirs, files in os.walk(delegate_options['artifacts_folder'], topdown=False):
    [os.remove(os.path.join(root, f)) for f in files]
    [os.rmdir(os.path.join(root, d)) for d in dirs]

input_image = calib_images


numFrames = config['num_images']

if numFrames > delegate_options['advanced_options:calibration_frames']:
    numFrames = delegate_options['advanced_options:calibration_frames']


c_interpreter = tflite.Interpreter(model_path=config['model_path'], \
                experimental_delegates=[tflite.load_delegate(os.path.join(tidl_tools_path, 'tidl_model_import_tflite.so'), delegate_options)])

# run Compilation
for i in range(numFrames):
    start_index = i%len(input_image)
    input_details = c_interpreter.get_input_details()
    batch = input_details[0]['shape'][0]
    input_images = []
    #for batch > 1 input images will be more than one in single input tensor
    for j in range(batch):
        input_images.append(input_image[(start_index+j)%len(input_image)])
    imgs, output, new_height, new_width  = infer_image(c_interpreter, input_images, config)
 

interpreter = tflite.Interpreter(model_path=config['model_path'], \
                        experimental_delegates=[tflite.load_delegate('libtidl_tfl_delegate.so', delegate_options)])

# run Inference
numFrames = 1
for i in range(numFrames):
    start_index = i%len(test_images)
    input_details = interpreter.get_input_details()
    batch = input_details[0]['shape'][0]
    input_images = []
    #for batch > 1 input images will be more than one in single input tensor
    for j in range(batch):
        input_images.append(test_images[(start_index+j)%len(test_images)])
    imgs, output, new_height, new_width  = infer_image(interpreter, input_images, config)


images = []
if config['model_type'] == 'classification':
    for j in range(batch):         
        classes, image = get_class_labels(output[0][j],imgs[j])
        images.append(image)
        print("\n", classes)
elif config['model_type'] == 'od':
    for j in range(batch):
        classes, image = det_box_overlay(output, imgs[j], config['od_type'])
        images.append(image)
elif config['model_type'] == 'seg':
    for j in range(batch):
        classes, image = seg_mask_overlay(output[0][j], imgs[j])
        images.append(image)
else:
    print("Not a valid model type")

for j in range(batch):
    output_file_name = "py_out_"+model+'_'+os.path.basename(input_images[j])
    print("\nSaving image to ", output_images_folder)
    if not os.path.exists(output_images_folder):
        os.makedirs(output_images_folder)
    images[j].save(output_images_folder + output_file_name, "JPEG")
